*COMPARATIVA PIBS DEL MUNDO*

In [2]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install pandas_datareader
!pip install yfinance   # si luego quieres probar con índices bursátiles
!pip install seaborn    # para mejorar la visualización (si prefieres)
!pip install pandas pandas_datareader matplotlib seaborn --quiet
!pip install pandas pandas_datareader plotly dash --quiet

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Construye una lista de países (los de gapminder de Plotly).

Crea 40 fechas trimestrales desde 2013.

Simula un PIB por país como una caminata aleatoria (random walk) alrededor de 1000. No son datos reales; sirven para probar la interfaz.

df queda con columnas: Fecha, <pais1>, <pais2>, ... y cada columna es la serie temporal de PIB simulado.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State
import dash


In [ ]:

# Generar datos de ejemplo
np.random.seed(42)
paises = sorted(px.data.gapminder()['country'].unique())
fechas = pd.date_range(start='2013-01-01', periods=40, freq='Q')

# Crear datos de PIB simulados por país
data = {pais: np.cumsum(np.random.randn(len(fechas))) * 10 + 1000 for pais in paises}
df = pd.DataFrame(data, index=fechas).reset_index().rename(columns={'index': 'Fecha'})

# Asignar colores distintos a cada país
def color_por_pais(pais):
    np.random.seed(abs(hash(pais)) % (2**32))
    r, g, b = np.random.randint(50, 256, 3)
    return f'rgb({r},{g},{b})'

colores_paises = {pais: color_por_pais(pais) for pais in paises}

# Crear la aplicación Dash
app = Dash(__name__)
app.title = "Mapa Geopolítico Interactivo de PIB"

# Layout principal de la aplicación
app.layout = html.Div([
    html.H2("Mapa geopolítico interactivo — PIB por país"),

    # Mapa del mundo arriba
    dcc.Graph(id='mapa-pib', style={'height': '600px'}),

    # Controles de comparación
    html.Div([
        html.Div([
            html.Label("Selecciona país 1:"),
            dcc.Dropdown(paises, id='pais1', value='Spain')
        ], style={'width': '45%', 'display': 'inline-block'}),

        html.Div([
            html.Label("Selecciona país 2:"),
            dcc.Dropdown(paises, id='pais2', value='France')
        ], style={'width': '45%', 'display': 'inline-block', 'marginLeft': '5%'})
    ]),

    html.Button('Correlacionar', id='btn-correlacion', n_clicks=0),

    html.Div(id='correlacion-output', style={'marginTop': '20px'}),

    dcc.Graph(id='grafico-pibs'),

    # Popup flotante para gráfico individual
    html.Div(id='popup', style={
        'display': 'none',
        'position': 'fixed',
        'top': '10%',
        'left': '10%',
        'width': '80%',
        'height': '70%',
        'backgroundColor': 'white',
        'border': '2px solid black',
        'boxShadow': '0px 4px 12px rgba(0,0,0,0.3)',
        'zIndex': 1000,
        'padding': '10px'
    }, children=[
        html.Button("Cerrar", id='cerrar-popup', style={'float': 'right', 'marginBottom': '10px'}),
        html.H3(id='titulo-pais', style={'textAlign': 'center'}),
        dcc.Graph(id='grafico-pib-pais', style={'height': '90%'})
    ])
])

# Callback para mostrar el mapa coloreado
@app.callback(
    Output('mapa-pib', 'figure'),
    Input('mapa-pib', 'clickData')
)
def mostrar_mapa(clickData):
    colores = [colores_paises[p] for p in paises]
    fig = px.choropleth(
        locations=paises,
        locationmode='country names',
        color=paises,
        color_discrete_sequence=colores,
        title="Mapa geopolítico mundial — haz clic en un país para ver su PIB histórico"
    )
    fig.update_layout(
        geo=dict(
            projection_type='natural earth',
            showcoastlines=True,
            showcountries=True,
            countrycolor="black"
        ),
        showlegend=False
    )
    return fig

# Callback para mostrar popup con gráfico del país clicado
@app.callback(
    Output('popup', 'style'),
    Output('grafico-pib-pais', 'figure'),
    Output('titulo-pais', 'children'),
    Input('mapa-pib', 'clickData'),
    Input('cerrar-popup', 'n_clicks'),
    prevent_initial_call=True
)
def mostrar_popup(clickData, cerrar_click):
    # Cerrar si se pulsa el botón
    ctx = dash.callback_context
    if not ctx.triggered:
        return dash.no_update, dash.no_update, dash.no_update

    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    if trigger_id == 'cerrar-popup':
        return {'display': 'none'}, go.Figure(), ""

    # Mostrar gráfico al hacer clic en un país
    pais = clickData['points'][0]['location']
    if pais not in df.columns:
        return {'display': 'none'}, go.Figure(), ""

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['Fecha'],
        y=df[pais],
        mode='lines+markers',
        line=dict(color='royalblue'),
        name=pais
    ))
    fig.update_layout(
        title=f"PIB histórico de {pais}",
        xaxis_title="Fecha",
        yaxis_title="PIB (valor simulado)",
        template="plotly_white"
    )

    return {'display': 'block'}, fig, f"PIB histórico de {pais}"

# Callback para comparar dos países y calcular la correlación
@app.callback(
    Output('correlacion-output', 'children'),
    Output('grafico-pibs', 'figure'),
    Input('btn-correlacion', 'n_clicks'),
    State('pais1', 'value'),
    State('pais2', 'value')
)
def comparar_paises(n_clicks, p1, p2):
    if not p1 or not p2 or p1 not in df.columns or p2 not in df.columns:
        return "Selecciona dos países válidos.", go.Figure()

    df_sel = df[['Fecha', p1, p2]]
    corr = df_sel[p1].pct_change().corr(df_sel[p2].pct_change())

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_sel['Fecha'], y=df_sel[p1], mode='lines', name=p1))
    fig.add_trace(go.Scatter(x=df_sel['Fecha'], y=df_sel[p2], mode='lines', name=p2))
    fig.update_layout(title=f'PIB de {p1} y {p2} en el tiempo')

    return f"Correlación entre {p1} y {p2}: {corr:.2f}", fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run(debug=True)


C:\Users\Asus Vivobook\AppData\Local\Temp\ipykernel_28520\2756098778.py:11: FutureWarning:

'Q' is deprecated and will be removed in a future version, please use 'QE' instead.



Esta normalizado incorrectamente